# Day 22: Slam Shuffle

https://adventofcode.com/2019/day/22#part2

## Part 1

Full-size brute-force shuffling using lists

In [1]:
import numpy as np

In [2]:
size = 10
deck = np.arange(size).reshape(size)
print(deck)

[0 1 2 3 4 5 6 7 8 9]


In [3]:
# 1. deal into new stack
def dealNew(deck):
    newdeck = deck[::-1] 
    return newdeck

print(dealNew(deck))

[9 8 7 6 5 4 3 2 1 0]


In [4]:
# 2. cut N cards (N can be negative)
def cutN(deck,N):
    if N>0:
        cut = deck[0:N]
        left = deck[N:]
        newdeck = np.append(left,cut)
    elif N<0:
        cut = deck[len(deck)+N:]
        left = deck[:N]
        newdeck = np.append(cut,left)
    return newdeck
    
print(cutN(deck,3))

[3 4 5 6 7 8 9 0 1 2]


In [5]:
# 3. deal with increment N
def dealWithIncrementN(deck,N):
    newdeck = np.zeros(len(deck))
    j = 0
    for i in range(len(deck)):
        #print(j)
        newdeck[j] = deck[i]
        j = (j+N)%len(deck)
    return newdeck
 
deck = np.arange(size).reshape(size)
print(dealWithIncrementN(deck,3))

[0. 7. 4. 1. 8. 5. 2. 9. 6. 3.]


In [6]:
lines = [
'deal with increment 7',
'deal into new stack',
'deal into new stack']
# Result: 0 3 6 9 2 5 8 1 4 7 --> OK

lines = [
'cut 6',
'deal with increment 7',
'deal into new stack' ] 
#Result: 3 0 7 4 1 8 5 2 9 6 --> OK

lines = [
'deal with increment 7',
'deal with increment 9',
'cut -2' ]
#Result: 6 3 0 7 4 1 8 5 2 9 --> OK

lines = [
'deal into new stack',
'cut -2',
'deal with increment 7',
'cut 8',
'cut -4',
'deal with increment 7',
'cut 3',
'deal with increment 9',
'deal with increment 3',
'cut -1' ]
# Result: 9 2 5 8 1 4 7 0 3 6 --> OK

size = 10

In [8]:
with open("./data/input22.txt") as f:
    lines = [l.rstrip('\n') for l in f]
#lines

size = 10007

In [9]:
s1 = 'deal into new stack'
s2 = 'cut '
s3 = 'deal with increment '

deck = np.arange(size).reshape(size)

for l in lines:
    if l == s1:
        deck = dealNew(deck)
    elif l[0:4]== s2:
        n = l.split(' ')
        N = int(n[1])
        deck = cutN(deck,N)
    else:
        n = l.split(' ')
        N = int(n[3])
        deck = dealWithIncrementN(deck,N)

In [10]:
card = 2019
pos = np.where(deck==card)[0][0]
print("Card",card,"is found at position",pos)

Card 2019 is found at position 7171


## Part 2

Direct shuffling is unfeasible given the new deck size. On the other hand, each shuffling operation can be represented as a **linear function** of the card value:

- deal: `(N-x-1) % N`
- cut: `(x-n) % N`
- deal with increment: `(x*n) % N`

The sequence of shufflings can therefore be expressed as a **combination of linear functions**.

In [11]:
with open("./data/input22.txt") as f:
    lines = [l.rstrip('\n') for l in f]

s1 = 'deal into new stack'
s2 = 'cut '
s3 = 'deal with increment '

seq = [] # save wha shuffling is operated, and, in case of cut and deal with increment, the value

for l in lines:
    if l == s1:
        seq.append([1,0])
    elif l[0:4]== s2:
        ll = l.split(' ')
        n = int(ll[1])
        seq.append([2,n])
    else:
        ll = l.split(' ')
        n = int(ll[3])
        seq.append([3,n])

In [12]:
# mod operation is factorizable!

def deal(x,N):
    return (N-x-1) #% N

def cut(x,N,n):
    return (x-n) #% N

def dealwi(x,N,n):
    return (x*n) #% N

def sequence(x,N,seq):
    for s in seq:
        if s[0]==1:
            x = deal(x,N)
        elif s[0]==2:
            x = cut(x,N,s[1])
        elif s[0]==3:
            x = dealwi(x,N,s[1])
    return x % N

In [13]:
x = 2019
N = 10007
print(sequence(x,N,seq))

7171


`sequence()` solves Part 1 by implementing all instructions as operations on a single card/value, and that's good. On the other hand, it's still defined as a sequence of operations, while I'd like to **condensate all operations in a simplified $y=ax+b$ form**. Since each operation is a linear tranformation (plus a `mod()` operation that is factorisable), I can compute the "cumulative" factors $a$ and $b$ factor as "sum" of the coefficients of all transformations ($a$'s needs to be multipled, $b$'s are summed, but I should not forget to also multiply the previous $b$ value if the current tranformation has a scaling factor $a$).

In [14]:
def linearSeq(seq,N):
    a = 1
    b = 0
    for s in seq:
        if s[0]==1:
            a *= -1
            b = (-1*b)+N-1 # b needs to be multiplied by -1 (a of current transforamtion) before shifting
        elif s[0]==2:
            #a *= 1
            b -= s[1]
        elif s[0]==3:
            a *= s[1]
            b *= s[1]
    return a%N,b%N

a, b = linearSeq(seq,N)
print("a =",a,"b =",b)

a = 5636 b = 6046


In [15]:
x = 2019
y = (a * x + b)%N
print("x =",x,"--> y =",y)

x = 2019 --> y = 7171


I now have a simple linear function $y = ax + b$ that solves Part 1! In order to solve Part 2, I need the inverse of this function to get $x = (y-b)/a$, but I cannot really divide by $a$ since I'm working with integer numbers and modular algebra. I therefore need to find the **multiplicative inverse** $1/a \mod N$. To do it I can use the **extended Euclid algorithm**: http://www.di-srv.unisa.it/~ads/ads/Sicurezza_files/NumberTheory.pdf


In [16]:
def EuclidGDC(a,n):
    '''Euclid algorithm to computed GCD'''
    if n==0:
        return a
    else:
        return EuclidGDC(n, a%n)

def ExtendedEuclidGDC(a,n): 
    '''Extended Euclid algorithm to computed GCD and integer pair (x,y) that satifies d = gcd(a,n) = a*x+n*y'''
    if n==0:
        return a,1,0
    di, xi, yi = ExtendedEuclidGDC(n,a%n)
    d = di
    x = yi
    y = xi - a//n * yi # use // for integer division!
    return d, x, y 
    

d,x,y = ExtendedEuclidGDC(99,78)
d,x,y
#=-11*99+14*78

(3, -11, 14)

I can solve an equation in the form $ax = n\mod n$ it with Euclid. Solutions exists if and only if $g|b$ where $g = \gcd(a,n)$. In this case I have $g$ solutions:
$$ x'\frac{b}{g} + i \frac{n}{g} \quad\text{for}\quad i = 0,1,..., g-1$$

In my case I want to invert $y=ax+b \mod n$, so I want to find $a^{-1}\mod{n}$ and $b/a\mod{n}$ to be able to compute:
$$ x = y \frac{1}{a} - \frac{b}{a}\mod n$$

In [17]:
# example: compute di 5^-1 mod 7
d,x,y = ExtendedEuclidGDC(5,7)
print("5^-1 mod 7 =",x)

5^-1 mod 7 = 3


In [18]:
D,inva,Y = ExtendedEuclidGDC(a,N)
print("a =", a,"--> 1/a mod N =",inva)

a = 5636 --> 1/a mod N = 3631


In [19]:
A = inva
B = - b * inva

y = 7171
x = (A*y+B)%N

print('y =',y,'--> x =',x)

y = 7171 --> x = 2019


I now want to apply the inverse function M = 101741582076661 times. Direct application is not feasible, but I can compute the "cumulative" transformation corresponding to iteratively applying the inverse transformation M times. 

Before doing this, I need to recompute the linear transformation parameters $a,b$ and $A,B$ for a deck of size N = 119315717514047:

In [20]:
N = 119315717514047 # size of deck

a, b = linearSeq(seq,N)
print("Linear transform a,b = ",a,b)
D,inva,Y = ExtendedEuclidGDC(a,N)
A = inva
B = (- b * inva)%N
print("Inverse linear transform A,B = ",A,B)

Linear transform a,b =  38251765999964 57940767209306
Inverse linear transform A,B =  14101252936313 37661399858180


Now I need to apply M = 101741582076661 times the inverse tranformation, that I defined as:

$$G_1(y) = ( Ay + B ) \mod N$$

with A and B computed as before. 

If I apply $G_1$ twice I get:

$$G_2(y) = G_1(G_1(y)) = A^2 + B (A+1) \mod N$$

Following the same procedure, I can define the generic $G_M$ transformation corresponding to applyting M times $G_1$ as:

$$G_M(y) = A^M + B \sum_{i=0}^{M-1} A^i \mod N$$

The first term $A^M \mod N$ requires a **modular exponentiation**. 

The second term contains a **geometric series**, for which I know the sum:

$$\sum_{i=0}^{M-1} A^i = \frac{1-A^M}{1-A}$$

I therefore get:

$$G_M(y) = A^M y + B \frac{A^M-1}{A-1} \mod N$$

Now the issue is that this is computed in a modular agebra context, so I cannot simply divide by $(A-1)$, but I need to **compute $1/(A-1)$ using the extended Euclid algorithm** as I did before.

In [21]:
def modpow(x, y, z):
    "Calculate (x ** y) % z"
    number = 1
    while y:
        if y & 1:
            number = number * x % z
        y >>= 1
        x = x * x % z
    return number

In [22]:
M = 101741582076661 # number of repetition of shuffling sequence

AM = modpow(A,M,N) # My implementation
#AM = pow(A,M,N) # Python bult-in function 

DD,inv1A,Y = ExtendedEuclidGDC((A-1),N)

BM = (B * (AM-1)*inv1A )%N

print("Cumulative inverse linear transform coefficients (AM, BM) =",AM,BM)

Cumulative inverse linear transform coefficients (AM, BM) = 113756250669691 87839588395182


In [23]:
Y = 2020
X = (AM*Y+BM)%N
print("Number on card ending in position",Y,"=",X)

Number on card ending in position 2020 = 73394009116480
